## Dependencies

In [ ]:
# %pip install datasets
# %pip install matplotlib

## Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import statistics

## Dataset

##### Import

In [78]:
from datasets import load_dataset

ds = load_dataset("hishab/hishab-pr-bn-v1")

In [79]:
ds

DatasetDict({
    train: Dataset({
        features: ['conversations', 'source', 'score'],
        num_rows: 2000
    })
})

In [80]:
df = ds["train"]
df

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 2000
})

##### Random

In [ ]:
df = ds["train"]
print(df)
# print(f"Features: {df.features}")

In [ ]:
for name, features in df.features.items():
    print(f"{name}: {features}")

In [ ]:
unique_conversation_len = {len(item) for item in df["conversations"]}
print(unique_conversation_len)
print({item[0]['from'] for item in df["conversations"]})
print({item[1]['from'] for item in df["conversations"]})

In [ ]:
# print random 5 "synthetic data" example
examples = df.filter(lambda x: x["source"] == "synthetic_data")
# print(examples.to_pandas()[:5])

examples = [x for x in df if x["source"] == "synthetic_data"][:5]
for x in examples:
    print(x)

##### Dropping the `score` column
* Quality score ranging from 0.0 to 10.0. This randomly generated value is a byproduct of the dataset processing pipeline and is not specifically relevant to the punctuation restoration task.


In [ ]:
# dropping "score" column
# score: Quality score ranging from 0.0 to 10.0. This randomly generated value is a byproduct of the dataset processing pipeline and is not specifically relevant to the punctuation restoration task.

if "score" in df.column_names:
    df = df.remove_columns(column_names="score")
df

In [ ]:
# scores = list(df["score"])

# plt.figure()
# plt.hist(scores)
# plt.xlabel("Score")
# plt.ylabel("Frequency")
# plt.title("Score Distribution")
# plt.show()



# word_lens = [
#     sum(len(turn["value"].split()) for turn in conv)
#     for conv in df["conversations"]
# ]

# plt.figure()
# plt.hist(word_lens)
# plt.xlabel("Total Words per Conversation")
# plt.ylabel("Frequency")
# plt.title("Conversation Length (Words)")
# plt.show()


# human_conv_len = [
#     len(conv[0]["value"].split())
#     for conv in df["conversations"]
# ]

# human_conv_len = np.array(human_conv_len)
# human_conv_len = np.sort(human_conv_len)
# print(human_conv_len)
# human_conv_len.min(), human_conv_len.max(), human_conv_len.mean()


# empty_human_indices = [
#     i for i, conv in enumerate(df["conversations"])
#     if len(conv[0]["value"].split()) <= 0
# ]

# print("Number of empty human inputs:", len(empty_human_indices))
# print("Indices of empty human inputs:", empty_human_indices)

# print(df[empty_human_indices[:]])

##### Analyzing

In [76]:
# number of unique source
df.unique("source")

['bd_news_data', 'synthetic_data', 'hqtts_data', 'non_verbal_data']

In [77]:
# row counts from each source
Counter(df["source"])

Counter({'synthetic_data': 1110,
         'bd_news_data': 833,
         'hqtts_data': 50,
         'non_verbal_data': 7})

In [90]:
# first row's "conversation" column
df["conversations"][0]

[{'from': 'human',
  'value': 'বাংলাদেশ প্রতিনিধিদলের একটি সূত্র অবশ্য প্রথম আলোকে জানায় তিস্তা নিয়ে কিছু বলার সময় এখনো আসেনি এ নিয়ে তাড়াহুড়োও নেই ভারতের আশ্বাসে আমরা সন্তুষ্ট মমতা বন্দ্যোপাধ্যায়ের ওপরেও আমাদের ভরসা আছে আমরা জানি ঠিক সময়েই এই বোঝাপড়া হয়ে যাবে চার দেশীয় যান চলাচল চুক্তি বিবিআইএন নিয়েও দুই পররাষ্ট্রসচিব বিস্তারিত আলোচনা করেছেন'},
 {'from': 'gpt',
  'value': 'বাংলাদেশ প্রতিনিধিদলের একটি সূত্র অবশ্য প্রথম আলোকে জানায়, তিস্তা নিয়ে কিছু বলার সময় এখনো আসেনি। এ নিয়ে তাড়াহুড়োও নেই। ভারতের আশ্বাসে আমরা সন্তুষ্ট। মমতা বন্দ্যোপাধ্যায়ের ওপরেও আমাদের ভরসা আছে। আমরা জানি ঠিক সময়েই এই বোঝাপড়া হয়ে যাবে, চার দেশীয় যান চলাচল চুক্তি বিবিআইএন নিয়েও দুই পররাষ্ট্রসচিব বিস্তারিত আলোচনা করেছেন।'}]

In [104]:
# number of entry in each conversation
# must be equal to 2
assert all(len(conv) == 2 for conv in df["conversations"]), "Must be exactly two entry"

In [109]:
# number of unique conversation participants
unique_participants = {
    participant['from'] for conv in df["conversations"]
    for participant in conv
}
assert len(unique_participants) == 2, "Must be only two differnt participants"
unique_participants

{'gpt', 'human'}

In [ ]:
# min, max, average conversation character length
conv_len = [
    len(entry["value"]) for conv in df["conversations"] for entry in conv
]
assert len(conv_len) == 4000, "Should be equal 4000"

conv_len = np.array(conv_len)
conv_len.min(), conv_len.max(), conv_len.mean()

(np.int64(0), np.int64(943), np.float64(138.56775))

In [139]:
# min, max, average conversation word length
conv_word_len = [len(entry["value"].split()) for conv in df["conversations"] for entry in conv]
conv_word_len = np.array(conv_word_len)
conv_word_len.min(), conv_word_len.max(), conv_word_len.mean()

(np.int64(0), np.int64(135), np.float64(22.642))

In [170]:
# minimum, maximum, average times puncuation added to a sentence
punc_added_counts = [
    len(conv[1 ]["value"]) - len(conv[0]["value"]) for conv in df["conversations"]
]

punc_added_counts = np.array(punc_added_counts)
punc_added_counts.min(), punc_added_counts.max(), punc_added_counts.mean()

(np.int64(0), np.int64(34), np.float64(4.1845))

In [181]:
# print punc added to a sentence
def get_punc_added(without_punc_senc, with_punc_senc):
    """
    Returns a string of characters in `second` that do not match `first`,
    filtered to only include punctuation.
    """
    punc_added = ""
    i = 0
    for ch in with_punc_senc:
        if i < len(without_punc_senc) and ch == without_punc_senc[i]:
            i = i + 1
        else:
            punc_added += ch
    return punc_added

added = get_punc_added(
    df["conversations"][0][0]["value"],
    df["conversations"][0][1]["value"]
)
print(added)

,।।।।,।


In [185]:
# distict punc list added
punc_list = [get_punc_added(conv[0]["value"], conv[1]["value"]) for conv in df["conversations"]]
distinct_puncs = set("".join(punc_list))
distinct_puncs

{' ',
 '!',
 '"',
 '%',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '?',
 '[',
 ']',
 '।',
 '—',
 '‘',
 '’'}

In [190]:
# number of times each punctuation occur
punc_counts = Counter("".join(punc_list))
punc_counts

Counter({'।': 3477,
         ',': 2543,
         '?': 1424,
         '"': 430,
         '!': 246,
         '-': 73,
         ')': 50,
         '(': 47,
         "'": 29,
         ' ': 23,
         '%': 17,
         '.': 5,
         '‘': 1,
         '’': 1,
         '—': 1,
         '[': 1,
         ']': 1})